In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import RidgeClassifier

In [2]:
file = '../Resampling/FCV_Oversampled.csv'
df = pd.read_csv(file)

df

,Previous PHEVs,Previous BEVs,Previous HEVs,Previous CNGs,Household Income,"Importance of reducing greenhouse gas emissions (-3 not important, 3 important)",Home ownership (own 1),Home Type (detached 1),Highest Level of Education,Longest trip in the last 12 months,...,One-way commute distance,Number of people in the household,Age,Gender (Male 1),Number of vehicles in the household,Annual VMT Estimate,Target,Model Year,Manufacturer,Model
0,0,0,0,0,125000.0,-2,1,1,3,571.75,...,15.58,2,55,0,3,10992,1.0,2013,4,4
1,0,0,1,0,225000.0,1,1,1,4,120.40,...,2.19,2,55,0,2,5714,1.0,2014,1,13
2,0,0,0,0,375000.0,2,1,1,3,407.45,...,40.95,5,45,0,2,16800,1.0,2013,9,6
3,0,0,1,0,275000.0,2,1,1,4,123.65,...,3.04,3,45,0,2,6000,1.0,2012,9,6
4,0,0,0,0,125000.0,0,1,1,4,289.51,...,47.75,2,65,0,2,14250,1.0,2014,12,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6629,0,0,1,0,275000.0,2,1,1,4,568.09,...,10.69,3,65,0,3,9197,0.0,2016,12,7
6630,0,0,1,0,225000.0,2,0,0,4,788.46,...,24.94,2,35,0,3,12800,0.0,2015,5,14
6631,0,0,0,0,225000.0,0,1,1,4,395.48,...,33.58,4,55,0,2,21600,0.0,2017,12,7
6632,0,0,0,0,75000.0,0,1,0,2,447.83,...,4.52,2,65,0,2,14880,0.0,2017,4,3


In [3]:
ml_df = df.copy()

In [5]:
# Seperate training data from target column

X = ml_df.drop(columns=('Target'), axis = 1)
y = ml_df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [6]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
rr = RidgeClassifier(alpha=1)

In [11]:
rr.fit(X_train_scaled, y_train)

RidgeClassifier(alpha=1)

In [15]:
y_pred = rr.predict(X_test_scaled)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test,
}).reset_index(drop=True)
results

,Prediction,Actual
0,1.0,1.0
1,0.0,0.0
2,0.0,0.0
3,1.0,1.0
4,0.0,0.0
...,...,...
1986,0.0,0.0
1987,1.0,0.0
1988,0.0,1.0
1989,1.0,1.0


In [16]:
# Evaluate the model
ovr_accuracy_score = accuracy_score(y_test, y_pred)
ovr_accuracy_score

0.8307383224510296